In [32]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [34]:

data = pd.read_csv("saving_model/dataset/Churn_Modelling.csv")


data.drop(labels=["RowNumber", "CustomerId", "Surname", "Exited"],axis=1,inplace=True)

X = data.drop("EstimatedSalary", axis=1)
y = data["EstimatedSalary"]


In [35]:
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

print("categorical column:", categorical_cols)
print("numerical column:", numerical_cols)



categorical column: Index(['Geography', 'Gender'], dtype='object')
numerical column: Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember'],
      dtype='object')


In [36]:
# Defining the categories for ordinal encoding
gender_cat = ['Female', 'Male']
geo_cat = ['France', 'Spain', 'Germany']

In [37]:
# Creating pipelines for numerical and categorical features
# Pipeline for numerical features
num_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
            

# Pipeline for categorical features
cat_pipeline = Pipeline(steps=[('ordinalencoder', OrdinalEncoder(categories=[geo_cat, gender_cat]))])
               # Convert categorical values into numbers

# Combine numerical and categorical pipelines

preprocessor = ColumnTransformer(
                    [("num_pipeline", num_pipeline, numerical_cols),      # Apply numerical pipeline to numerical columns
                     ("cat_pipeline", cat_pipeline, categorical_cols)]    # Apply categorical pipeline to categorical columns
                )

In [38]:
# Preprocessing
X_processed = preprocessor.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42
)


In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [40]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.001)


In [41]:
# ANN REGRESSION MODEL
model = Sequential([
    Dense(64, activation='relu', input_shape = (X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(
    optimizer=opt,
    loss="mse"
)


In [42]:
# Train
model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32
)


Epoch 1/50
200/200 [==============================] - 2s 5ms/step - loss: 13307445248.0000 - val_loss: 13690853376.0000
Epoch 2/50
200/200 [==============================] - 0s 1ms/step - loss: 13204943872.0000 - val_loss: 13468863488.0000
Epoch 3/50
200/200 [==============================] - 0s 1ms/step - loss: 12815517696.0000 - val_loss: 12865485824.0000
Epoch 4/50
200/200 [==============================] - 0s 1ms/step - loss: 12012444672.0000 - val_loss: 11815097344.0000
Epoch 5/50
200/200 [==============================] - 0s 2ms/step - loss: 10796807168.0000 - val_loss: 10382137344.0000
Epoch 6/50
200/200 [==============================] - 0s 2ms/step - loss: 9300564992.0000 - val_loss: 8765577216.0000
Epoch 7/50
200/200 [==============================] - 0s 2ms/step - loss: 7760546304.0000 - val_loss: 7230656512.0000
Epoch 8/50
200/200 [==============================] - 0s 2ms/step - loss: 6427561984.0000 - val_loss: 6018183168.0000
Epoch 9/50
200/200 [==========================

In [43]:
# Evaluation
y_pred = model.predict(X_test).flatten()

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R2 Score:", r2)


63/63 [==============================] - 0s 906us/step
MSE: 3336098950.817789
RMSE: 57758.97290307186
MAE: 49916.51210296874
R2 Score: -0.010555287158572568


In [44]:
# Save model & preprocessor
model.save("model.h5")

with open("preprocessor.pkl", "wb") as f:
    pickle.dump(preprocessor, f)


c:\Users\Asus\.conda\envs\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
